# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [109]:
!pip install pyspark
from pyspark.sql import SparkSession
import numpy as np

spark =SparkSession.builder.appName('Spark Exercise ').getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [110]:
df_walmart=spark.read.format('csv').options(inferSchem=True,Header=True).load('/content/walmart_stock.csv')

#### What are the column names?

In [111]:
df_walmart.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [112]:
df_walmart.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



#### Print out the first 5 columns.

In [128]:
df_walmart.head(5)

[Row(Date='2012-01-03', Open=Decimal('59.97'), High=Decimal('61.06'), Low=Decimal('59.87'), Close=Decimal('60.33'), Volume=Decimal('12668800.00'), Adj Close=Decimal('52.62')),
 Row(Date='2012-01-04', Open=Decimal('60.21'), High=Decimal('60.35'), Low=Decimal('59.47'), Close=Decimal('59.71'), Volume=Decimal('9593300.00'), Adj Close=Decimal('52.08')),
 Row(Date='2012-01-05', Open=Decimal('59.35'), High=Decimal('59.62'), Low=Decimal('58.37'), Close=Decimal('59.42'), Volume=Decimal('12768200.00'), Adj Close=Decimal('51.83')),
 Row(Date='2012-01-06', Open=Decimal('59.42'), High=Decimal('59.45'), Low=Decimal('58.87'), Close=Decimal('59.00'), Volume=Decimal('8069400.00'), Adj Close=Decimal('51.46')),
 Row(Date='2012-01-09', Open=Decimal('59.03'), High=Decimal('59.55'), Low=Decimal('58.92'), Close=Decimal('59.18'), Volume=Decimal('6679300.00'), Adj Close=Decimal('51.62'))]

#### Use describe() to learn about the DataFrame.

In [114]:
df_walmart.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|         10010500|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [115]:
### The cast column used below to update the schema directly
from pyspark.sql.functions import round, col
describe_df=df_walmart.describe()
cols=df_walmart.columns
for col_name in cols:
  describe_df=describe_df.withColumn(col_name,col(col_name).cast('decimal(10,2)'))

describe_df.show()

+-------+-------+-------+-------+-------+-------+-----------+---------+
|summary|   Date|   Open|   High|    Low|  Close|     Volume|Adj Close|
+-------+-------+-------+-------+-------+-------+-----------+---------+
|  count|1258.00|1258.00|1258.00|1258.00|1258.00|    1258.00|  1258.00|
|   mean|   null|  72.36|  72.84|  71.92|  72.39| 8222093.48|    67.24|
| stddev|   null|   6.77|   6.77|   6.74|   6.76| 4519780.84|     6.72|
|    min|   null|  56.39|  57.06|  56.30|  56.42|10010500.00|    50.36|
|    max|   null|  90.80|  90.97|  89.25|  90.47| 9994400.00|    84.91|
+-------+-------+-------+-------+-------+-------+-----------+---------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [116]:
new_df=df_walmart.withColumn('HV Ratio',df_walmart.High/df_walmart.Volume)
new_df.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [117]:
# in case original data itself is to be updated with correct schema
from datetime import datetime

cols=df_walmart.columns[1:]
for i in cols:
  df_walmart=df_walmart.withColumn(i,col(i).cast('decimal(12,2)'))
df_walmart.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: decimal(12,2) (nullable = true)
 |-- High: decimal(12,2) (nullable = true)
 |-- Low: decimal(12,2) (nullable = true)
 |-- Close: decimal(12,2) (nullable = true)
 |-- Volume: decimal(12,2) (nullable = true)
 |-- Adj Close: decimal(12,2) (nullable = true)



####  ***What day had the Peak High in Price?**

In [118]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
from pyspark.sql.functions import udf

date_conversion= udf(lambda x: datetime.strptime(x,'%Y-%m-%d'), TimestampType())

df_walmart.orderBy('High',ascending=0).select(date_conversion('Date')).first().asDict()


{'<lambda>(Date)': datetime.datetime(2015, 1, 13, 0, 0)}

#### What is the mean of the Close column?

In [119]:

df_walmart.agg({'Close':'mean'}).show()

+----------+
|avg(Close)|
+----------+
| 72.388450|
+----------+



#### What is the max and min of the Volume column?

In [120]:
df_walmart.agg(max("Volume"),min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|80898100.00| 2094900.00|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [121]:
df_walmart.filter('Close < 60').agg(count('Date')).show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [122]:
df_walmart.filter('High < 60')

DataFrame[Date: string, Open: decimal(12,2), High: decimal(12,2), Low: decimal(12,2), Close: decimal(12,2), Volume: decimal(12,2), Adj Close: decimal(12,2)]

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [123]:
df_walmart.agg(corr('High','Volume')).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.33843260582148915|
+--------------------+



#### What is the max High per year?

In [127]:
df_walmart.groupBy(year('Date').alias('Year')).max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|    77.60|
|2016|    75.19|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [126]:
df_walmart.groupBy(month('Date').alias('Month')).agg({'Close':'mean'}).orderBy('Month').show()

+-----+----------+
|Month|avg(Close)|
+-----+----------+
|    1| 71.448020|
|    2| 71.306804|
|    3| 71.777944|
|    4| 72.973619|
|    5| 72.309717|
|    6| 72.495377|
|    7| 74.439720|
|    8| 73.029818|
|    9| 72.184118|
|   10| 71.578545|
|   11| 72.111089|
|   12| 72.847925|
+-----+----------+



# Great Job!